In [0]:
#retrieve details for Azure Account
storage_account_name = "iaksuts"

In [0]:
storage_account_access_key = "HySt59PJVmS0OL08etJ+kGS3shiZVf3Kxq4wHdTqzgmXls3oa6EkCwWJDp/KxLcdmup8kW8jf9oe+ASti6isPQ=="

In [0]:
blob_container_name = "bde-at-2"

In [0]:
#Mount all files from Azure to Databricks
dbutils.fs.mount(
  source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
  mount_point = f'/mnt/{blob_container_name}/',
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-185319031148975> in <module>
----> 1 dbutils.fs.mount(
      2   source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
      3   mount_point = f'/mnt/{blob_container_name}/',
      4   extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
      5 )

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    387                     exc.__context__ = None
    388                     exc.__cause__ = None
--> 389                     raise exc
    390 
    391             return f_with_exception_handling

ExecutionError: An error occurred while calling o401.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/bde-at-2; nested exc

In [0]:
#List all files in stored directory
dbutils.fs.ls("/mnt/bde-at-2/")

Out[4]: [FileInfo(path='dbfs:/mnt/bde-at-2/converted_yellow_april22/', name='converted_yellow_april22/', size=0, modificationTime=1663145728000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-01.parquet', name='green_tripdata_2019-01.parquet', size=11071980, modificationTime=1663140044000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-02.parquet', name='green_tripdata_2019-02.parquet', size=10316792, modificationTime=1663140031000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-03.parquet', name='green_tripdata_2019-03.parquet', size=10760872, modificationTime=1663140070000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-04.parquet', name='green_tripdata_2019-04.parquet', size=9412998, modificationTime=1663140033000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-05.parquet', name='green_tripdata_2019-05.parquet', size=9050672, modificationTime=1663140034000),
 FileInfo(path='dbfs:/mnt/bde-at-2/green_tripdata_2019-06.parquet', name='green_t

In [0]:
#Read green taxi trip data
green_df = spark.read.parquet("/mnt/bde-at-2/green_tripdata_*.parquet")

In [0]:
#Read yellow taxi trip data
yellow_df = spark.read.parquet("/mnt/bde-at-2/yellow_tripdata_*.parquet")

In [0]:
yellow_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-03-01 00:24:41|  2019-03-01 00:25:31|            1.0|          0.0|       1.0|                 N|         145|         145|           2|        2.5|  0.5|    0.5|       0.

In [0]:
#Retrieve number of rows
yellow_df.count()

In [0]:
#Original number of rows = 152,823,008

#Removal of 0 passenger / empty cabs = 146,699,968 

#Removal of negative trip_distances/zero distance = 145,199,410

#Removal of fares < $2.50 (initial surcharge) = 144,524,779

#Removal of cabs that have travelled at a speed greater than 55 mph = 144,430,886

#Removal of rows with pickup time after the dropoff time = 144,429,165

#Removal of rows with cabs having more than 5 passengers = 141,322,770

#Removal of trips with a distance greater than 150 miles (too far) = 141,322,664

#Removal of trips with a duration greater than 2.5 hours (too long) = 140,988,886

#Removal of trips with distance travelled less than 500m = 139,198,910

#Removal of trips with a duration less than 2 minutes (too short) = 138,140,317

#Removal of trips with speed less than 1MPH = 138,133,360

In [0]:
#Retrieve number of rows
green_df.count()

Out[16]: 9390483

In [0]:
#Create subset of April2022 data for conversion
yellow_april = spark.read.parquet("/mnt/bde-at-2/yellow_tripdata_2022-04.parquet")
#yellow_april.write.csv('/mnt/bde-at-2/converted_yellow_april22',mode='append')

In [0]:
#Filter out rows with empty cabs/no passengers
yellow_df = yellow_df.filter(yellow_df.passenger_count != 0)

In [0]:
#Filter out rows where distance travelled is 0
yellow_df = yellow_df.filter(yellow_df.trip_distance > 0)

In [0]:
#Filter out negative/less than minimum fare and total amounts
yellow_df = yellow_df.filter(yellow_df.fare_amount > 2.50) 

In [0]:
yellow_df = yellow_df.filter(yellow_df.total_amount > 2.50)

In [0]:
#Filter out rows where cabs have travelled at a speed greater than 55 mph using formula (SPEED = DISTANCE/TIME)
yellow_df = yellow_df.filter(yellow_df.trip_distance/((yellow_df.tpep_dropoff_datetime.cast("long") - yellow_df.tpep_pickup_datetime.cast("long"))/3600) < 55.95)

In [0]:
#Filter out rows that have a pickup time after the dropoff
yellow_df = yellow_df.filter(yellow_df.tpep_dropoff_datetime > yellow_df.tpep_pickup_datetime)

In [0]:
#Filter out rows with passengers > 5
yellow_df = yellow_df.filter(yellow_df.passenger_count <= 5)

In [0]:
#Filter out rows where distance travelled is greater than 150 miles (becomes an interstate trip)
yellow_df = yellow_df.filter(yellow_df.trip_distance < 150)

In [0]:
#Filter out trips that are longer than 2.5 hours
yellow_df = yellow_df.filter((yellow_df.tpep_dropoff_datetime.cast("long") - yellow_df.tpep_pickup_datetime.cast("long"))/3600 < 2.5)

In [0]:
#Filter out rows where distance travelled is less than 500m (0.310 miles)
yellow_df = yellow_df.filter(yellow_df.trip_distance > 0.310)

In [0]:
#Filter out rows that are shorter than 2 minutes
yellow_df = yellow_df.filter((yellow_df.tpep_dropoff_datetime.cast("long") - yellow_df.tpep_pickup_datetime.cast("long"))/60 > 2)

In [0]:
#Filter out rows where cabs have travelled at a speed less than 1 mph using formula (SPEED = DISTANCE/TIME)
yellow_df = yellow_df.filter(yellow_df.trip_distance/((yellow_df.tpep_dropoff_datetime.cast("long") - yellow_df.tpep_pickup_datetime.cast("long"))/3600) > 1)

In [0]:
#Original number of rows = 9,390,483

#Removal of 0 passenger / empty cabs = 7,728,604

#Removal of negative trip_distances/zero distance = 7,489,210

#Removal of fares < $2.50 (initial surcharge) = 7,418,217

#Removal of cabs that have travelled at a speed greater than 55 mph = 7,407,839

#Removal of rows with pickup time after the dropoff time = 7,407,823

#Removal of rows with cabs having more than 5 passengers = 7,287,208

#Removal of trips with a distance greater than 150 miles (too far) = 7,287,201

#Removal of trips with a duration greater than 3 hours (too long) = 7,246,943

#Removal of trips with distance travelled less than 500m = 7,154,338

#Removal of trips with a duration less than 2 minutes (too short) = 7,108,866

#Removal of trips with speed less than 1MPH = 7,107,941

In [0]:
#Applying the same filtering conditions to the green table dataset as well

In [0]:
#Retrieve number of rows
green_df.count()

Out[88]: 7107941

In [0]:
#Filter out rows with empty cabs/no passengers
green_df = green_df.filter(green_df.passenger_count >= 0)

In [0]:
#Filter out rows where distance travelled is 0
green_df = green_df.filter(green_df.trip_distance > 0)

In [0]:
#Filter out negative/less than minimum fare and total amounts
green_df = green_df.filter(green_df.fare_amount > 2.50) 

In [0]:
green_df = green_df.filter(green_df.total_amount > 2.50)

In [0]:
#Filter out rows where cabs have travelled at a speed greater than 55 mph using formula (SPEED = DISTANCE/TIME)
green_df = green_df.filter(green_df.trip_distance/((green_df.lpep_dropoff_datetime.cast("long") - green_df.lpep_pickup_datetime.cast("long"))/3600) < 55.95)

In [0]:
#Filter out rows that have a pickup time after the dropoff
green_df = green_df.filter(green_df.lpep_dropoff_datetime > green_df.lpep_pickup_datetime)

In [0]:
#Filter out rows with passengers > 5
green_df = green_df.filter(green_df.passenger_count <= 5)

In [0]:
#Filter out rows where distance travelled is greater than 150 miles (becomes an interstate trip)
green_df = green_df.filter(green_df.trip_distance < 150)

In [0]:
#Filter out trips that are longer than 2.5 hours
green_df = green_df.filter((green_df.lpep_dropoff_datetime.cast("long") - green_df.lpep_pickup_datetime.cast("long"))/3600 < 2.5)

In [0]:
#Filter out rows where distance travelled is less than 500m (0.310 miles)
green_df = green_df.filter(green_df.trip_distance > 0.310)

In [0]:
#Filter out rows that are shorter than 2 minutes
green_df = green_df.filter((green_df.lpep_dropoff_datetime.cast("long") - green_df.lpep_pickup_datetime.cast("long"))/60 > 2)

In [0]:
#Filter out rows where cabs have travelled at a speed less than 1 mph using formula (SPEED = DISTANCE/TIME)
green_df = green_df.filter(green_df.trip_distance/((green_df.lpep_dropoff_datetime.cast("long") - green_df.lpep_pickup_datetime.cast("long"))/3600) > 1)

In [0]:
green_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- taxi_colour: string (nullable = false)



In [0]:
yellow_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- taxi_colour: string (nullable = false)



In [0]:
#Change payment_type column to double in yellow_df
from pyspark.sql.types import DoubleType
yellow_df = yellow_df.withColumn("payment_type",yellow_df.payment_type.cast(DoubleType()))

In [0]:
#Adding taxi_colour column required for analysis
from pyspark.sql.functions import lit
green_df = green_df.withColumn("taxi_colour",lit("Green"))
yellow_df = yellow_df.withColumn("taxi_colour",lit("Yellow"))

In [0]:
#Renaming columns of yellow_Taxi prior to merge for convenience
yellow_df = yellow_df.withColumnRenamed("tpep_pickup_datetime","lpep_pickup_datetime")
yellow_df = yellow_df.withColumnRenamed("tpep_dropoff_datetime","lpep_dropoff_datetime")

In [0]:
#merging yellow and green taxi dataframes
merged_df = yellow_df.unionByName(green_df, allowMissingColumns=True)

In [0]:
merged_df.count()

Out[78]: 143403154

In [0]:
#Extracting dayofweek from dataset
from pyspark.sql.functions import date_format
merged_df = merged_df.withColumn("dayofweek", date_format(merged_df.lpep_pickup_datetime,'EEEE'))

In [0]:
#Extract hour from pickup time
from pyspark.sql.functions import hour
merged_df = merged_df.withColumn('hourofday', hour(merged_df.lpep_pickup_datetime))

In [0]:
#Extracting month and year column from the dataset
from pyspark.sql.functions import year, month
merged_df = merged_df.withColumn("month", month(merged_df.lpep_pickup_datetime))
merged_df = merged_df.withColumn("tripyear", year(merged_df.lpep_pickup_datetime))

In [0]:
#Filtering records in merged datasets with weird times (eg: year=2088)

In [0]:
merged_df = merged_df.filter(merged_df.tripyear >= 2019)

In [0]:
merged_df = merged_df.filter(merged_df.tripyear <= 2022)

In [0]:
#removing vendorID that is not part of the data dictionary
merged_df = merged_df.filter(merged_df.VendorID != 4)

In [0]:
#removing RatecodeID that is not part of the data dictionary
merged_df = merged_df.filter(merged_df.RatecodeID != 99)

In [0]:
#removing PULocationID that is not part of the data dictionary
merged_df = merged_df.filter(merged_df.PULocationID < 264)

In [0]:
#removing DOLocationID that is not part of the data dictionary
merged_df = merged_df.filter(merged_df.DOLocationID < 264) 

In [0]:
#Creating a temporary view to perform analysis
merged_df.createOrReplaceTempView("merged_table")

In [0]:
#Creating a sample of data to perform small anlysis on
sample_df = merged_df.sample(0.0002)

In [0]:
sample_df.createOrReplaceTempView("sample_table")

In [0]:
sample_df.count()

Out[72]: 28950

In [0]:
merged_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)
 |-- taxi_colour: string (nullable = false)
 |-- ehail_fee: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- dayof

In [0]:
spark.sql('''
SELECT * FROM
sample_table

LIMIT 10
'''
).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+---------+---------+---------+---------+-----+--------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|taxi_colour|ehail_fee|trip_type|dayofweek|hourofday|month|tripyear|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+---------+---------+---------+---------+--

In [0]:
spark.sql('''

WITH cte_base AS (
SELECT
    concat(tripyear,'-',month) AS year_month,
    COUNT(*) AS total_trips,
    ROUND(AVG(passenger_count),2) AS avg_number_passengers,
    ROUND(AVG(total_amount),2) AS avg_amount_per_trip,
    ROUND(AVG(total_amount/passenger_count),2) AS avg_amount_per_passenger
FROM merged_table
GROUP BY 1
),
cte_day_of_week AS (
SELECT
    concat(tripyear,'-',month) AS year_month,
    dayofweek,
    COUNT(*) AS day_trip_count,
    ROW_NUMBER() OVER (
        PARTITION BY concat(tripyear,'-',month)
        ORDER BY COUNT(dayofweek) DESC
    ) AS trip_rank
FROM merged_table
GROUP BY concat(tripyear,'-',month), dayofweek
),
cte_hour_of_day AS (
SELECT
    concat(tripyear,'-',month) AS year_month,
    hourofday,
    COUNT(*) AS hour_trip_count,
    ROW_NUMBER() OVER (
        PARTITION BY concat(tripyear,'-',month)
        ORDER BY COUNT(hourofday) DESC
    ) AS trip_rank
FROM merged_table
GROUP BY concat(tripyear,'-',month), hourofday
)
-- final output
SELECT
    cte_base.year_month,
    cte_base.total_trips,
    cte_day_of_week.dayofweek,
    cte_day_of_week.day_trip_count,
    cte_hour_of_day.hourofday,
    cte_hour_of_day.hour_trip_count,
    cte_base.avg_number_passengers,
    cte_base.avg_amount_per_trip,
    cte_base.avg_amount_per_passenger
FROM cte_base
LEFT JOIN cte_day_of_week
    ON cte_base.year_month = cte_day_of_week.year_month
    AND cte_day_of_week.trip_rank = 1
LEFT JOIN cte_hour_of_day
    ON cte_base.year_month = cte_hour_of_day.year_month
    AND cte_hour_of_day.trip_rank = 1
ORDER BY cte_base.year_month
    
'''
         ).show()

+----------+-----------+---------+--------------+---------+---------------+---------------------+-------------------+------------------------+
|year_month|total_trips|dayofweek|day_trip_count|hourofday|hour_trip_count|avg_number_passengers|avg_amount_per_trip|avg_amount_per_passenger|
+----------+-----------+---------+--------------+---------+---------------+---------------------+-------------------+------------------------+
|    2019-1|    7431115| Thursday|       1310918|       18|         502927|                 1.46|              15.69|                   13.34|
|   2019-10|    6914535| Thursday|       1204472|       18|         467483|                 1.46|              19.34|                   16.31|
|   2019-11|    6575222|   Friday|       1163620|       18|         437727|                 1.46|              19.02|                   16.02|
|   2019-12|    6579228|  Tuesday|       1037709|       18|         422326|                 1.47|              19.17|                   16.03|

In [0]:
#Question 2
spark.sql('''

WITH taxi_colour_cte AS(
SELECT taxi_colour,
    ROUND(AVG(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)),2) AS avg_trip_duration,
    MIN(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)) AS min_trip_duration,
    MAX(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)) AS max_trip_duration,
    percentile_approx(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime), 0.5) AS median_trip_duration,
    ROUND(AVG(trip_distance)*1.609,2) AS avg_distance_km,
    ROUND(MIN(trip_distance)*1.609,2) AS min_distance_km,
    ROUND(MAX(trip_distance)*1.609,2) AS max_distance_km,
    ROUND(percentile_approx(trip_distance, 0.5)*1.609,2) AS median_distance_km,
    ROUND(AVG(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60))*1.609, 2) AS avg_speed_km,
    ROUND(MIN(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60))*1.609,2) AS min_speed_km,
    ROUND(MAX(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60))*1.609,2) AS max_speed_km,
    ROUND(percentile_approx(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60),0.5)*1.609,2) AS median_speed_km
    
    
    FROM merged_table
    GROUP BY taxi_colour

)
SELECT *
FROM taxi_colour_cte




'''
).show()

+-----------+-----------------+-----------------+-----------------+--------------------+---------------+---------------+---------------+------------------+------------+------------+------------+---------------+
|taxi_colour|avg_trip_duration|min_trip_duration|max_trip_duration|median_trip_duration|avg_distance_km|min_distance_km|max_distance_km|median_distance_km|avg_speed_km|min_speed_km|max_speed_km|median_speed_km|
+-----------+-----------------+-----------------+-----------------+--------------------+---------------+---------------+---------------+------------------+------------+------------+------------+---------------+
|      Green|            14.37|                2|              149|                  11|            5.1|           0.51|         169.86|              3.12|       20.57|        1.62|      122.12|           18.5|
|     Yellow|            13.76|                2|              149|                  11|           4.87|           0.51|         205.31|              2.75| 

In [0]:
#Question 3
spark.sql('''

WITH tip_calc_cte AS(
SELECT
    SUM (
        CASE
            WHEN tip_amount > 0 THEN 1
            ELSE 0
        END) AS paid_tips,
    COUNT(1) AS total_trips
    
FROM merged_table
)

SELECT ROUND(paid_tips/total_trips*100,2) AS percent_tipped_trips
FROM tip_calc_cte



'''



).show()

+--------------------+
|percent_tipped_trips|
+--------------------+
|               70.48|
+--------------------+



In [0]:
#Question 4
spark.sql('''

WITH tip_calc_cte AS(
SELECT
    SUM (
        CASE
            WHEN tip_amount >= 10 THEN 1
            ELSE 0
        END) AS paid_tips,
    COUNT(1) AS total_trips
    
FROM merged_table
WHERE tip_amount > 0
)

SELECT ROUND(paid_tips/total_trips*100,2) AS percent_tipped_trips_over_10
FROM tip_calc_cte



'''



).show()

+----------------------------+
|percent_tipped_trips_over_10|
+----------------------------+
|                        3.41|
+----------------------------+



In [0]:
#Question 5
spark.sql('''
WITH classify_cte AS(
SELECT
    CASE
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) < 5 THEN 'Under 5'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 5 AND 10 THEN 'Between 5 & 10'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 10 AND 20 THEN 'Between 10 & 20'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 20 AND 30 THEN 'Between 20 & 30'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 30 AND 60 THEN 'Between 30 & 60'
        ELSE 'Atleast 60'
    END as time_buckets, 
    trip_distance, lpep_pickup_datetime, lpep_dropoff_datetime, total_amount
    
FROM merged_table       
)

SELECT 
time_buckets,
ROUND(AVG(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60))*1.609, 2) AS avg_speed_km,
ROUND(AVG(trip_distance*1.609/total_amount), 2) AS avg_distance_per_dollar
FROM classify_cte
GROUP BY time_buckets
ORDER BY avg_distance_per_dollar
    

''').show()

+---------------+------------+-----------------------+
|   time_buckets|avg_speed_km|avg_distance_per_dollar|
+---------------+------------+-----------------------+
|        Under 5|       22.85|                   0.13|
| Between 5 & 10|       18.05|                   0.18|
|Between 10 & 20|       18.36|                   0.23|
|Between 20 & 30|       22.44|                   0.29|
|Between 30 & 60|       27.56|                   0.35|
|     Atleast 60|       23.09|                    0.4|
+---------------+------------+-----------------------+



In [0]:
#Question 6
spark.sql('''
WITH classify_cte AS(
SELECT
    CASE
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) < 5 THEN 'Under 5'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 5 AND 10 THEN 'Between 5 & 10'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 10 AND 20 THEN 'Between 10 & 20'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 20 AND 30 THEN 'Between 20 & 30'
        WHEN DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime) BETWEEN 30 AND 60 THEN 'Between 30 & 60'
        ELSE 'Atleast 60'
    END as time_buckets, 
    trip_distance, lpep_pickup_datetime, lpep_dropoff_datetime, total_amount
    
FROM merged_table       
)

SELECT 
time_buckets,
COUNT(*) as trips_per_time_bucket,
ROUND(AVG(trip_distance/(DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/60))*1.609, 2) AS avg_speed_km,
ROUND(AVG(total_amount/trip_distance*1.609), 2) AS avg_amount_per_km
FROM classify_cte
GROUP BY time_buckets
ORDER BY trips_per_time_bucket

''').show()

+---------------+---------------------+------------+-----------------+
|   time_buckets|trips_per_time_bucket|avg_speed_km|avg_amount_per_km|
+---------------+---------------------+------------+-----------------+
|     Atleast 60|              1060946|       23.09|             7.03|
|Between 30 & 60|              9398571|       27.56|             8.34|
|Between 20 & 30|             15991205|       22.44|             10.4|
|        Under 5|             18138065|       22.85|            21.93|
|Between 10 & 20|             46871564|       18.36|            12.79|
| Between 5 & 10|             51942803|       18.05|            16.37|
+---------------+---------------------+------------+-----------------+



In [0]:
#Regression Modelling 
#1. Linear Regression

In [0]:
#new_df = merged_df.sample(0.0002)
new_df = merged_df.filter((merged_df.tripyear != 2022) & (merged_df.month != 4))

In [0]:
#Conversion of Taxi Colour column from Categorical to Nominal
from pyspark.ml.feature import StringIndexer, OneHotEncoder
indexer = StringIndexer(inputCol='taxi_colour', outputCol='colour_numeric').fit(new_df)

In [0]:
new_df = indexer.transform(new_df)

In [0]:
#Selecting columns of interest
new_df = new_df.select("total_amount", "VendorID", "RatecodeID", "trip_distance", "tip_amount", "mta_tax", "tolls_amount", "passenger_count", "PULocationID", "DOLocationID","extra","colour_numeric")

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [0]:
#Using vector Assembler to combine all features
vector_4 = VectorAssembler(inputCols=['VendorID', 'RatecodeID', 'trip_distance', 'tip_amount', 'mta_tax', 'tolls_amount', 'passenger_count', 'PULocationID', 'DOLocationID','extra','colour_numeric'], outputCol='features')

In [0]:
#(train_data, test_data) = new_df.randomSplit([0.8, 0.2])
train_data = new_df
test_data = merged_df.filter((merged_df.tripyear == 2022) & (merged_df.month == 4))

In [0]:
vector_data = vector_4.transform(train_data)

In [0]:
vector_data.show()

+------------+--------+----------+-------------+----------+-------+------------+---------------+------------+------------+-----+--------------------+
|total_amount|VendorID|RatecodeID|trip_distance|tip_amount|mta_tax|tolls_amount|passenger_count|PULocationID|DOLocationID|extra|            features|
+------------+--------+----------+-------------+----------+-------+------------+---------------+------------+------------+-----+--------------------+
|         3.8|       1|       1.0|          0.4|       0.0|    0.5|         0.0|            1.0|         151|         151|  0.0|[1.0,1.0,0.4,0.0,...|
|         3.8|       2|       1.0|         0.33|       0.0|    0.5|         0.0|            1.0|         226|         226|  0.0|[2.0,1.0,0.33,0.0...|
|         3.8|       2|       1.0|         0.36|       0.0|    0.5|         0.0|            1.0|          42|          42|  0.0|[2.0,1.0,0.36,0.0...|
|         3.8|       2|       1.0|         0.39|       0.0|    0.5|         0.0|            1.0|    

In [0]:
#Creating new dataset with features and label column
features_data = vector_data.select(["features", "total_amount"])

In [0]:
lr = LinearRegression(labelCol="total_amount", featuresCol="features")

In [0]:
#Training the model
model = lr.fit(features_data)

In [0]:
summary = model.summary
print(summary.r2)
#r2 on sample size data WITH taxi_colour = 0.9502934591455252
#r2 on sample size data WITHOUT taxi_colour = 0.004260845821092318
#r2 on full dataset WITHOUT taxi_colour = 0.0063272197827577425
#r2 on full dataset WITH taxi_colour = 0.006329953183734305

0.006329953183734305


In [0]:
#Instantiating regression evaluator
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="total_amount", metricName="rmse")

In [0]:
#Conversion of Taxi Colour column from Categorical to Nominal
from pyspark.ml.feature import StringIndexer, OneHotEncoder
indexer = StringIndexer(inputCol='taxi_colour', outputCol='colour_numeric').fit(test_data)

In [0]:
test_data = indexer.transform(test_data)

In [0]:
vector_test = vector_4.transform(test_data)

In [0]:
features_test = vector_test.select(["features", "total_amount"])

In [0]:
test_transform = model.transform(features_test)

In [0]:
#Calculating RMSE
evaluator.evaluate(test_transform)
#rmse on sample size data WITH taxi_colour = 2.909756047491609
#rmse on sample size data WITHOUT taxi_colour = 4.4049782246613844
#rmse on entire dataset WITHOUT taxi_colour = 4.831286445467734
#rmse on entire dataset WITH taxi_colour = 4.8256521424625065

Out[61]: 4.8256521424625065

In [0]:
#Regression Modelling 
#2. Decision Tree Regressor

In [0]:
import pandas as pd
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'total_amount')

In [0]:
# instantiate and train
dt_model = dt.fit(features_data)

In [0]:
dt_test_transform = dt_model.transform(features_test)

In [0]:
#Calculating RMSE
evaluator.evaluate(dt_test_transform)

Out[89]: 5.298364401808545

In [0]:
dt_model.featureImportances

Out[90]: SparseVector(11, {1: 0.0002, 2: 0.0178, 3: 0.0012, 4: 0.0, 5: 0.0004, 7: 0.966, 8: 0.0, 9: 0.0145})

In [0]:
train_data.take(1)

Out[91]: [Row(total_amount=15.0, VendorID=1, RatecodeID=1.0, trip_distance=3.7, tip_amount=0.7, mta_tax=0.5, tolls_amount=0.0, passenger_count=2.0, PULocationID=95, DOLocationID=130, extra=0.5, colour_numeric=0.0)]